## <font face="仿宋">课程说明：

- 体验课内容节选自[《2025大模型Agent智能体开发实战》(秋招冲刺班)](https://ix9mq.xetslk.com/s/2S2Vpy)完整版付费课程

&emsp;&emsp;体验课时间有限，若想深度学习大模型技术，欢迎大家报名由我主讲的[《2025大模型Agent智能体开发实战》(秋招冲刺班)](https://ix9mq.xetslk.com/s/2S2Vpy)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/b3a518f1a9821408a79363cf694f5172.jpg" alt="b3a518f1a9821408a79363cf694f5172" style="zoom:33%;" />

**[《2025大模型Agent智能体开发实战》(秋招冲刺班)](https://ix9mq.xetslk.com/s/2S2Vpy)为【100+小时】体系大课，总共20大模块精讲精析，零基础直达大模型企业级应用！**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202506172010074.png" alt="a55d48e952ed59f8d93e050594843bc" style="zoom:33%;" />

部分项目成果演示

In [9]:
from IPython.display import Video

- **MateGen项目演示**

In [4]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/MG%E6%BC%94%E7%A4%BA%E8%A7%86%E9%A2%91.mp4", width=800, height=400)

- **智能客服项目演示**

In [8]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E6%99%BA%E8%83%BD%E5%AE%A2%E6%9C%8D%E6%A1%88%E4%BE%8B%E8%A7%86%E9%A2%91.mp4", width=800, height=400)

- **Dify项目演示**

In [6]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/2f1b47f42c65fd59e8d3a83e6cb9f13b_raw.mp4", width=800, height=400)

- **LangChain&LangGraph搭建Multi-Agnet**

In [7]:
Video("https://ml2022.oss-cn-hangzhou.aliyuncs.com/%E5%8F%AF%E8%A7%86%E5%8C%96%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90Multi-Agent%E6%95%88%E6%9E%9C%E6%BC%94%E7%A4%BA%E6%95%88%E6%9E%9C.mp4", width=800, height=400)

此外，若是对大模型底层原理感兴趣，也欢迎报名由我和菜菜老师共同主讲的[《2025大模型原理与实战课程》(秋招冲刺班)](https://ix9mq.xetslk.com/s/3AME7R)

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/aaf3bafd8ff8120d5fb079f092268961.png" alt="aaf3bafd8ff8120d5fb079f092268961" style="zoom:33%;" />

**两门大模型课程秋招冲刺班预售进行中，直播间可享超值特价+全套学习福利，合购还有更多优惠哦~<span style="color:red;">详细信息扫码添加助教，回复“大模型”，即可领取课程大纲&查看课程详情👇</span>**

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/26449c9c3e90ea66e0af9150ad00e0c6.png" alt="26449c9c3e90ea66e0af9150ad00e0c6" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/bff56b8959b8a17d9d0a6d1e72f5d9b8.png" alt="bff56b8959b8a17d9d0a6d1e72f5d9b8" style="zoom:50%;" />

---

# <center>GPT-OSS高效微调实战</center>
# <center>Part 4.GPT-OSS Agent性能微调</center>

&emsp;&emsp;在熟悉了Unsloth核心功能以及基础的微调数据集准备方法之后，接下来我们尝试进行一项更加复杂的微调工作——围绕GPT-OSS模型进行Agent性能微调。

## 九、Agent性能微调数据集准备

### 1. Hermes tool use数据集介绍

- 数据集地址：https://huggingface.co/datasets/interstellarninja/hermes_reasoning_tool_use

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250825191926108.png" alt="image-20250825191926108" style="zoom:50%;" />

&emsp;&emsp;**interstellarninja / hermes_reasoning_tool_use** 是一个围绕“会思考、会用工具”的开源数据集：采用 Apache-2.0 许可、单一 `train` 切分约 **51,004** 条样本（约 392 MB），以 ShareGPT 会话格式存储（Parquet），字段含 `conversations / tools / task / category / source / scenario_category`，便于直接用  Datasets/axolotl 加载微调。其对话模板在 `system` 中明确要求**将内在推理写入 `<think>…</think>`**，并以 `<tool_call>…</tool_call>` 输出 **OpenAI 风格的函数调用 JSON（含 pydantic schema）**，非常契合“思考链 + 函数调用”的监督信号。
 样本覆盖 BFCL v3 的四类场景：**single_turn**、**multi_turn**、**multi_step** 与 **relevance**（无合适工具时应拒绝），有助于训练模型“何时/如何/是否”调用工具。构建流程基于 **Nous Research 的 Atropos RL 栈**，在自定义 `MultiTurnToolCallingEnv` 中进行 **GRPO** roll-outs，并通过奖励塑形与严格校验（如 `_validate_think_plus_calls`、`_check_sequential_tools`）仅保留结构与语义均合格的轨迹。

&emsp;&emsp;并且，数据集包含四种调用格式，`single_turn`（单轮单调用）、`multi_turn`（多轮多调用）、`multi_step`（单轮内**≥2 顺序调用**）、`relevance`（**无合适工具应拒绝**）。这天然覆盖了“何时/如何/是否”调用工具的决策学习。

&emsp;&emsp;总体而言，这是一个**兼具长链推理标注与工具调用轨迹**的实用数据集，既可用于 **SFT**，也可作为 **GRPO 预热**或评测基座，快速提升模型在真实代理场景下的工具使用稳健性。

- 数据集读取

In [18]:
from datasets import load_dataset, Dataset, Features, Value
import random, json, os

In [19]:
# 原始数据
ds_full = load_dataset("interstellarninja/hermes_reasoning_tool_use", split="train")

Using the latest cached version of the dataset since interstellarninja/hermes_reasoning_tool_use couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/interstellarninja___hermes_reasoning_tool_use/default/0.0.0/55d824b623303055d5a76eb6ab12861b80a4ee20 (last modified on Mon Aug 25 20:12:37 2025).


In [23]:
ds_full[0]

{'conversations': [{'from': 'system',
   'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Here are the available tools: <tools> [{"name": "calculate_age", "description": "Calculates the age based on the birthdate.", "parameters": {"birthdate": {"description": "The birthdate in the format \'YYYY-MM-DD\'.", "type": "str"}}}, {"name": "find_duplicate_number", "description": "Finds the duplicate number in a list of integers

Hermes Reasoning Tool Use 是一个 **“推理 + 工具调用（function calling）”** 的教学/微调数据集。它不仅包含自然语言对话，还显式标注了：

* 系统提示（system）里提供的**工具签名**（函数名、参数等）；
* 模型在对话中**调用工具**的动作（函数名 + 参数）；
* 工具的**返回结果**（带 `<tool_response>`）；
* 模型的**思维过程**（使用 `<think>…</think>` 包裹的内在推理）。

因此它特别适合训练/评估：
**何时**调用哪个工具、**如何**构造参数、**如何**把工具结果融入最终答复、以及**多步工具调用**的策划与执行。

In [4]:
ds_full

Dataset({
    features: ['conversations', 'tools', 'task', 'category', 'source', 'scenario_category'],
    num_rows: 51004
})

In [13]:
ds_full['task'][:10]

["What is the word frequency of 'The quick brown fox jumps over the lazy dog'? And can you reverse the words in the sentence 'Hello world this is a test'?",
 "Search for 'laptops' on Tmall and retrieve the second page of results.",
 'I would like to extract details from a LinkedIn company page. Could you assist me in fetching the information from this URL: https://www.linkedin.com/company/abc-corporation?',
 'I need to compare the weights of two groups of athletes. Group X weighs [70, 72, 74, 76, 78] kg and Group Y weighs [71, 73, 75, 77, 79] kg. Perform the t-test with a significance level of 0.10.',
 'I\'d like to learn Spanish and need to understand my current proficiency level. Also, once I know my level, I need a translation of the following text: "I want to learn Spanish quickly" into Spanish.',
 'Retrieve the metadata information for Ethereum (ETH) on the Binance exchange, including metrics related to the asset.',
 'Can you perform a static analysis on floors 3, 7, and 11?',
 'H

In [11]:
ds_full['source'][:10]

['Salesforce-Xlam',
 'Nvidia-When2Call',
 'ToolAce',
 'Nvidia-When2Call',
 'ToolAce',
 'Nvidia-When2Call',
 'Nvidia-When2Call',
 'Nvidia-When2Call',
 'ToolAce',
 'Nvidia-When2Call']

In [12]:
ds_full['scenario_category'][:10]

['single',
 'single',
 'multiturn',
 'single',
 'multistep',
 'relevance',
 'relevance',
 'relevance',
 'relevance',
 'single']

数据集每条样本通常包含以下顶层字段：

* `conversations`：**对话轮次**（list）。每轮有：

  * `from`：`system` / `human` / `gpt` / `tool`
  * `value`：该轮文本。

    * system 轮里常见 `<tools> … </tools>`（提供函数签名）
    * gpt 轮里可能含 `<think>…</think>` 与一个或多个 `<tool_call> … </tool_call>`
    * tool 轮里含 `<tool_response> … </tool_response>`
* `tools`：工具清单（有时是 **字符串化 JSON**，有时是 list\[dict]）。与 system 里的 `<tools>` 一致或互补。
* `task`：该样本的人类任务简述。
* `category`：任务类别（如 `Is Hotel Available`、`Tool Use`、`Get Stock Price` 等）。
* `source`：来源（如 `Nvidia-When2Call`、`ToolAce`、`Glaive`、`Nous-Hermes`、`Salesforce-Xlam`）。
* `scenario_category`：场景类型：

  * `single`：单轮对话 + 单次或多次调用（在一轮内发起多个工具调用）
  * `multiturn`：多轮往返
  * `multistep`：需要按顺序完成多个工具步骤
  * `relevance`：强调选择**相关**工具（干扰项较多）

> ⚠️ 真实数据里 **类型并不总统一**：
>
> * `tools` 可能是字符串，也可能是对象数组；
> * `parameters.type` 可能写成 `"dict"`（理想应为 `"object"`）；
> * `required` 可能放在工具**外层**而不是 `parameters.required`；
> * 有的属性缺失 `description`。
>   这就是我们清洗规范化要解决的。

- 单工具调用示例

In [6]:
ds_full[0]['conversations']

[{'from': 'system',
  'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Here are the available tools: <tools> [{"name": "calculate_age", "description": "Calculates the age based on the birthdate.", "parameters": {"birthdate": {"description": "The birthdate in the format \'YYYY-MM-DD\'.", "type": "str"}}}, {"name": "find_duplicate_number", "description": "Finds the duplicate number in a list of integers.", "parameters": {

- 多工具并行调用示例

In [7]:
ds_full[1]['conversations']

[{'from': 'system',
  'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. If available tools are not relevant in assisting with user query, just respond in natural conversational language. Don\'t make assumptions about what values to plug into functions. After calling & executing the functions, you will be provided with function results within <tool_response> </tool_response> XML tags. Here are the available tools:\n<tools>\n[\n{"name": "products_v2_list", "description": "Fetches a list 

- 多轮对话+工具调用示例

In [8]:
ds_full[2]['conversations']

[{'from': 'system',
  'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. If available tools are not relevant in assisting with user query, just respond in natural conversational language. Don\'t make assumptions about what values to plug into functions. After calling & executing the functions, you will be provided with function results within <tool_response> </tool_response> XML tags. Here are the available tools:\n<tools>\n[{"name":"extract","description":"Extracts data from a LinkedIn

- 无干预下多轮工具调用示例

In [10]:
ds_full[4]['conversations']

[{'from': 'system',
  'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. If available tools are not relevant in assisting with user query, just respond in natural conversational language. Don\'t make assumptions about what values to plug into functions. After calling & executing the functions, you will be provided with function results within <tool_response> </tool_response> XML tags. Here are the available tools:\n<tools>\n[{"name":"get_language_proficiency","description":"Retrieve lan

### 2. Hermes tool use数据集清洗

In [5]:
import re, json, ast
from copy import deepcopy
from datasets import load_dataset

In [6]:
# ========= 1) 类型映射：把简写类型转为 JSON Schema =========
def _map_type(t: str):
    t = (t or "").strip()
    if t.lower() in ("str", "string"):
        return {"type": "string"}
    if t.lower() in ("int", "integer"):
        return {"type": "integer"}
    if t.lower() in ("float", "number"):
        return {"type": "number"}
    if t.lower() in ("bool", "boolean"):
        return {"type": "boolean"}
    if t.startswith("List[") and t.endswith("]"):
        inner = t[5:-1].strip()
        inner_schema = _map_type(inner)
        return {"type": "array", "items": {"type": inner_schema.get("type", "string")}}
    # 兜底
    return {"type": "string"}

In [7]:
import json, ast, re

# 1) 兜底的类型映射
def _map_base_type(t: str) -> str:
    t = (t or "string")
    if isinstance(t, str):
        t = t.strip().lower()
    else:
        t = str(t).strip().lower()
    # 去掉 like "bool, optional"
    t = t.split(",")[0].strip()
    MAP = {
        "str": "string", "string": "string",
        "int": "integer", "integer": "integer",
        "float": "number", "number": "number",
        "bool": "boolean", "boolean": "boolean",
        "dict": "object", "object": "object",
        "array": "array", "list": "array",
    }
    return MAP.get(t, "string")

# 2) 将各种 parameters 归一化为 JSON Schema
def _normalize_parameters_to_jsonschema(params) -> dict:
    if not isinstance(params, dict):
        return {"type": "object", "properties": {}}

    if params.get("type") in ("object", "dict") and "properties" in params:
        schema = dict(params)
        if schema["type"] == "dict":
            schema["type"] = "object"
        props = {}
        for p, spec in (schema.get("properties") or {}).items():
            if not isinstance(spec, dict):
                spec = {}
            base = _map_base_type(spec.get("type", "string"))
            props[p] = {
                "type": base,
                "description": spec.get("description", ""),  # ← 必有描述
            }
        req = schema.get("required")
        if not isinstance(req, list):
            req = list(props.keys())
        return {"type": "object", "properties": props, "required": req}

    # 简写 -> 正规化
    props, required = {}, []
    for p, spec in (params.items() if isinstance(params, dict) else []):
        if not isinstance(spec, dict):
            spec = {}
        base = _map_base_type(spec.get("type", "string"))
        props[p] = {
            "type": base,
            "description": spec.get("description", ""),  # ← 必有描述
        }
        required.append(p)
    return {"type": "object", "properties": props, "required": required}

# 3) Hermes tools → gpt-oss tools（超稳健版）
def hermes_tools_to_gptoss_tools(hermes_tools):
    out = []
    if not hermes_tools:
        return out

    for t in hermes_tools:
        if isinstance(t, str):
            try:
                t = json.loads(t)
            except Exception:
                try:
                    import ast
                    t = ast.literal_eval(t)
                except Exception:
                    continue
        if not isinstance(t, dict):
            continue

        fn = t["function"] if (t.get("type") == "function" and isinstance(t.get("function"), dict)) else t
        name = (fn.get("name") or "").strip()
        if not name:
            continue

        desc = fn.get("description") or ""  # ← 必有 description
        params = fn.get("parameters") or {}
        schema = _normalize_parameters_to_jsonschema(params)

        # ---- 补齐 properties 里的 type/description ----
        if schema.get("type") == "object" and isinstance(schema.get("properties"), dict):
            for p, spec in list(schema["properties"].items()):
                if not isinstance(spec, dict):
                    spec = {}
                spec.setdefault("type", "string")
                spec.setdefault("description", "")
                schema["properties"][p] = spec
        # ---------------------------------------------

        out.append({
            "type": "function",
            "function": {
                "name": name,
                "description": desc,
                "parameters": schema,
            },
        })
    return out

In [8]:
# ========= 3) 从 system 文本里提取/去掉 <tools>... 块 =========
_TOOLS_BLOCK_RE = re.compile(r"<tools>(.*?)</tools>", re.DOTALL)

def strip_tools_block(text: str) -> str:
    if not text:
        return text
    return _TOOLS_BLOCK_RE.sub("", text).strip()

def extract_tools_from_system(text: str):
    """尽力从 <tools>...</tools> 中解析出 Python 列表或 JSON 列表；失败则返回 []."""
    if not text:
        return []
    m = _TOOLS_BLOCK_RE.search(text)
    if not m:
        return []
    raw = m.group(1).strip()
    # 尝试 JSON
    try:
        return json.loads(raw)
    except Exception:
        pass
    # 尝试 Python 字面量（Hermes 常用单引号）
    try:
        val = ast.literal_eval(raw)
        return val if isinstance(val, list) else []
    except Exception:
        return []

In [9]:
# ========= 4) 解析 assistant 的 <think> 与 <tool_call> =========
_THINK_RE = re.compile(r"<think>(.*?)</think>", re.DOTALL)
_TOOLCALL_RE = re.compile(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", re.DOTALL)

def parse_assistant_content(value: str):
    """返回 (thinking:str|None, tool_calls:list[{'function':{'name', 'arguments'}}])"""
    if not value:
        return None, []
    thinking = None
    m = _THINK_RE.search(value)
    if m:
        thinking = m.group(1).strip()

    calls = []
    for m in _TOOLCALL_RE.finditer(value):
        blob = m.group(1)
        try:
            obj = json.loads(blob)
        except Exception:
            # 单引号风格
            try:
                obj = ast.literal_eval(blob)
            except Exception:
                continue
        name = obj.get("name")
        args = obj.get("arguments", {})
        if not isinstance(args, dict):
            # 再试一次解析
            try:
                args = json.loads(args) if isinstance(args, str) else {}
            except Exception:
                args = {}
        if name:
            calls.append({"function": {"name": name, "arguments": args}})
    return thinking, calls

In [10]:
# ========= 5) 解析 tool 的 <tool_response> JSON 内容为字符串 =========
_TOOLRESP_RE = re.compile(r"<tool_response>\s*(\{.*?\})\s*</tool_response>", re.DOTALL)

def parse_tool_response_value(value: str) -> str:
    """
    输入 Hermes 的 tool.value，输出一个 JSON 字符串（尽量规范化）。
    """
    if not value:
        return ""
    m = _TOOLRESP_RE.search(value)
    raw = (m.group(1).strip() if m else value.strip())
    # 尝试 JSON
    try:
        obj = json.loads(raw)
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        pass
    # 尝试 Python 字面量
    try:
        obj = ast.literal_eval(raw)
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        # 留原文（让模板做 tojson），至少是字符串
        return raw

In [11]:
DEV_NOISE_PATTERNS = [
    r"Here are the available tools:.*",
    r"Use the following pydantic model.*",
    r"Each function call should be enclosed.*",
    r"You are a function calling AI model\..*",
    r"within\s*<tools>\s*</tools>\s*XML tags.*",
]
DEV_NOISE_RE = re.compile("|".join(DEV_NOISE_PATTERNS), re.DOTALL)

def clean_developer_text(s: str) -> str:
    s = _TOOLS_BLOCK_RE.sub("", s or "")  # 先去掉 <tools>…</tools>
    s = DEV_NOISE_RE.sub("", s).strip()
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s


In [12]:
import re

# 原有噪音模式
DEV_NOISE_PATTERNS = [
    r"Here are the available tools:.*",
    r"Use the following pydantic model.*",
    r"Each function call should be enclosed.*",
    r"You are a function calling AI model\..*",
    r"within\s*<tools>\s*</tools>\s*XML tags.*",
    # 新增：清理 <think> 相关说明
    r"You should enclose.*?<think>.*?</think>.*?(solution|response).*",
    r"enclose your thoughts.*?<think>.*?</think>.*",
    r"internal monologue.*?<think>.*?</think>.*",
]
DEV_NOISE_RE = re.compile("|".join(DEV_NOISE_PATTERNS), re.DOTALL | re.IGNORECASE)

def final_clean_developer_text(s: str) -> str:
    """
    清理 Hermes system 提示，去掉工具声明、pydantic schema、<think> 标签说明，
    只保留干净的通用 developer 提示。
    """
    if not s:
        return ""
    # 先去掉 <tools>...</tools> 块
    s = _TOOLS_BLOCK_RE.sub("", s)
    # 去掉定义的噪音段落
    s = DEV_NOISE_RE.sub("", s).strip()
    # 压缩多余空行
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s


In [13]:
import re, json

_UNPAIRED_THINK_OPEN  = re.compile(r"<think>\s*", re.IGNORECASE)
_UNPAIRED_THINK_CLOSE = re.compile(r"\s*</think>", re.IGNORECASE)

def _strip_unpaired_think_tags(text: str) -> str:
    if not isinstance(text, str):
        return "" if text is None else str(text)
    # 去掉单独出现（或未配对）的 <think> / </think> 标签
    text = _UNPAIRED_THINK_OPEN.sub("", text)
    text = _UNPAIRED_THINK_CLOSE.sub("", text)
    return text

def _drop_none_fields(msg: dict) -> dict:
    # 删掉值为 None 的键
    return {k: v for k, v in msg.items() if v is not None}

def sanitize_messages_for_gptoss(messages: list) -> list:
    cleaned = []
    for m in messages:
        m = dict(m)  # 复制一份
        role = m.get("role")

        # 1) 去掉 thinking=None
        if "thinking" in m and (m["thinking"] is None or m["thinking"] == ""):
            m.pop("thinking", None)

        # 2) 对 content 做兜底清洗（移除残留 think 标签）
        if "content" in m:
            m["content"] = _strip_unpaired_think_tags(m["content"])

        # 3) tool 消息的 content 必须是字符串
        if role == "tool":
            c = m.get("content", "")
            if not isinstance(c, str):
                try:
                    m["content"] = json.dumps(c, ensure_ascii=False)
                except Exception:
                    m["content"] = str(c)

        # 4) assistant+tool_calls 情况：不能带 content
        if role == "assistant" and "tool_calls" in m and m.get("tool_calls"):
            m.pop("content", None)  # 保守起见清掉

        # 5) 删除 None 字段
        m = _drop_none_fields(m)

        # 6) 丢弃完全空的消息（极少数脏数据）
        if role and (m.get("content") or m.get("tool_calls") or m.get("thinking") or role == "tool"):
            cleaned.append(m)

    return cleaned


In [14]:
# ========= 6) 单条样本转换：Hermes → gpt-oss (messages + tools) =========
def hermes_to_gptoss_example(ex: dict):
    conv = ex.get("conversations") or []
    top_tools_raw = ex.get("tools")  # 可能是 str / list[str] / list[dict] / dict / None

    messages = []
    idx = 0
    sys_text = ""

    if len(conv) and conv[0].get("from") == "system":
        sys_text = conv[0].get("value", "") or ""
        # 若顶层无 tools，从 system 的 <tools>... 抽
        if not top_tools_raw:
            extracted = extract_tools_from_system(sys_text)
            if extracted:
                top_tools_raw = extracted

        dev_text = final_clean_developer_text(sys_text)
        if dev_text.strip():
            messages.append({"role": "developer", "content": dev_text.strip()})
        idx = 1

    # 1) 工具归一化（你已有）
    top_tools_norm = parse_tools_any(top_tools_raw)
    gptoss_tools = hermes_tools_to_gptoss_tools(top_tools_norm or [])

    # 2) 主循环（加一点点健壮性）
    for turn in conv[idx:]:
        who = (turn.get("from") or "").lower()
        val = turn.get("value", "")
        if not isinstance(val, str):  # 防御：确保是字符串
            val = str(val)

        if who in ("human", "user"):
            if val.strip():
                messages.append({"role": "user", "content": val})

        elif who in ("gpt", "assistant"):
            thinking, calls = parse_assistant_content(val)
            if calls:
                for j, call in enumerate(calls):
                    msg = {"role": "assistant", "tool_calls": [call]}
                    if j == 0 and thinking:   # 只有第一条带 thinking
                        msg["thinking"] = thinking
                    messages.append(msg)
            else:
                # 非工具调用回合
                content = val
                if thinking is not None:
                    content = _THINK_RE.sub("", val).strip()
                messages.append(
                    {"role": "assistant", **({"thinking": thinking} if thinking else {}), "content": content}
                )

        elif who == "tool":
            tool_payload = parse_tool_response_value(val)  # -> JSON 字符串
            messages.append({"role": "tool", "content": tool_payload})

    # 3) 收尾清理（你已有）
    messages = sanitize_messages_for_gptoss(messages)
    return {"messages": messages, "tools": gptoss_tools}

In [15]:
# ========= 7) 批量映射 + 渲染成 text（用于 SFT） =========
def convert_hermes_dataset_to_gptoss_text(ds, tokenizer, reasoning_effort="low"):
    """
    输入：datasets.Dataset（Hermes），以及已加载好的 gpt-oss tokenizer
    输出：新的 Dataset，包含：
      - messages（供调试）
      - tools_obj（供调试）
      - text（喂给 SFTTrainer 的列）
    """
    def map_one(ex):
        r = hermes_to_gptoss_example(ex)
        return {"messages": r["messages"], "tools_obj": r["tools"]}

    ds2 = ds.map(map_one)

    def render_batch(batch):
        out = []
        for msgs, tools in zip(batch["messages"], batch["tools_obj"]):
            txt = tokenizer.apply_chat_template(
                msgs,
                tools=tools,                   # ← 关键：工具通过参数传入
                add_generation_prompt=False,   # 训练集通常 False
                reasoning_effort=reasoning_effort,  # "low"/"medium"/"high"
                tokenize=False,
            )
            out.append(txt)
        return {"text": out}

    ds3 = ds2.map(render_batch, batched=True)
    return ds3

In [16]:
import json, ast

def parse_tools_any(obj):
    """
    把 Hermes 的 tools 字段归一化为 [dict, ...]：
    - 支持 str（JSON 或 Python 字面量，含单引号）
    - 支持 dict（单个工具，转成 [dict]）
    - 支持 list（其中元素可为 str 或 dict）
    - 其他类型返回 []
    """
    if obj is None:
        return []

    # 纯字符串：尝试 json -> ast
    if isinstance(obj, str):
        s = obj.strip()
        if not s:
            return []
        for loader in (json.loads, ast.literal_eval):
            try:
                val = loader(s)
                return parse_tools_any(val)  # 递归归一化
            except Exception:
                pass
        return []  # 实在解析不了就空

    # 字典：包成单元素列表
    if isinstance(obj, dict):
        return [obj]

    # 列表：逐个元素解析
    if isinstance(obj, list):
        out = []
        for it in obj:
            if isinstance(it, dict):
                out.append(it)
            elif isinstance(it, str):
                it_s = it.strip()
                ok = None
                for loader in (json.loads, ast.literal_eval):
                    try:
                        val = loader(it_s)
                        ok = val
                        break
                    except Exception:
                        pass
                if isinstance(ok, dict):
                    out.append(ok)
                elif isinstance(ok, list):
                    out.extend(parse_tools_any(ok))
                # 其他情况丢弃这个元素
        return out

    # 其他类型丢弃
    return []


定义好了数据清洗函数之后，接下来测试能否将原始数据转化为gpt-oss模型能够用于训练的文本格式：

In [20]:
ds_full

Dataset({
    features: ['conversations', 'tools', 'task', 'category', 'source', 'scenario_category'],
    num_rows: 51004
})

In [27]:
# 2) 随便取一条（比如第 0 条）
sample = ds_full[0]
print("原始 Hermes 数据：")
print(sample)

原始 Hermes 数据：
{'conversations': [{'from': 'system', 'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Here are the available tools: <tools> [{"name": "calculate_age", "description": "Calculates the age based on the birthdate.", "parameters": {"birthdate": {"description": "The birthdate in the format \'YYYY-MM-DD\'.", "type": "str"}}}, {"name": "find_duplicate_number", "description": "Finds the duplicate number in a list 

In [28]:
# 3) 单条转换（不用整个 map）
converted = hermes_to_gptoss_example(sample)
print("\n转换后的 gpt-oss messages：")
for m in converted["messages"]:
    print(m)


转换后的 gpt-oss messages：
{'role': 'developer', 'content': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.'}
{'role': 'user', 'content': "What is the word frequency of 'The quick brown fox jumps over the lazy dog'? And can you reverse the words in the sentence 'Hello world this is a test'?"}
{'role': 'assistant', 'tool_calls': [{'function': {'name': 'word_frequency', 'arguments': {'text': 'The quick brown fox jumps over the lazy dog'}}}], 'thinking': 'Okay, the user has two separate questions here. The first is to find the word frequency of the sentence "The quick brown fox jumps over the lazy dog". The second is to reverse the words in the sentence "Hello world this is a test". Let me break down each part step by step.\n\nStarting with the word frequency: I need to use the word_frequency tool. The input text is gi

In [1]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/unsloth/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
max_seq_length = 8192
dtype = None

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./gpt-oss-20b-bf16",
    dtype = dtype, 
    max_seq_length = max_seq_length, 
    load_in_4bit = True,  
    full_finetuning = False,
)

==((====))==  Unsloth 2025.8.7: Fast Gpt_Oss patching. Transformers: 4.56.0.dev0.
   \\   /|    NVIDIA H800 PCIe. Num GPUs = 1. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gpt_Oss does not support SDPA - switching to eager!


Loading checkpoint shards: 100%|██████████| 9/9 [00:06<00:00,  1.30it/s]


In [29]:
# 4) 渲染成 gpt-oss 文本
text = tokenizer.apply_chat_template(
    converted["messages"],
    tools=converted["tools"],
    add_generation_prompt=False,
    reasoning_effort="low",
    tokenize=False,
)
print("\n渲染后的训练文本：")


渲染后的训练文本：


In [30]:
text

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Calculates the age based on the birthdate.\ntype calculate_age = (_: {\n// The birthdate in the format \'YYYY-MM-DD\'.\nbirthdate: string\n}) => any;\n\n// Finds the duplicate number in a list of integers.\ntype find_duplicate_number = (_: {\n// The list of integers.\nnums: number[]\n}) => any;\n\n// Calculates the frequency of each word in 

In [44]:
sample = ds[15]
print("原始 Hermes 数据：")
print(sample)

原始 Hermes 数据：
{'conversations': [{'from': 'system', 'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. If available tools are not relevant in assisting with user query, just respond in natural conversational language. Don\'t make assumptions about what values to plug into functions. After calling & executing the functions, you will be provided with function results within <tool_response> </tool_response> XML tags. Here are the available tools:\n<tools>\n[{"name":"get_biodegradation_rate

In [45]:
converted = hermes_to_gptoss_example(sample)
print("\n转换后的 gpt-oss messages：")
for m in converted["messages"]:
    print(m)


转换后的 gpt-oss messages：
{'role': 'developer', 'content': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.'}
{'role': 'user', 'content': 'I am conducting an environmental impact study involving several substances. Could you help me find out the toxicity level of lead and then measure the air quality in New York City for the next 60 minutes based on the presence of lead?\n\n'}
{'role': 'assistant', 'tool_calls': [{'function': {'name': 'get_toxicity_level', 'arguments': {'substance': 'lead'}}}], 'thinking': "Okay, the user wants to find the toxicity level of lead and measure air quality in NYC for 60 minutes. Let me start with the first task: getting the toxicity level of lead. I'll use the 'get_toxicity_level' tool with the substance 'lead'.\n\nOnce I get that result, they also want air quality measurements in NYC f

In [65]:
text = tokenizer.apply_chat_template(
    converted["messages"],
    tools=converted["tools"],
    add_generation_prompt=False,
    reasoning_effort="low",
    tokenize=False,
)
print("\n渲染后的训练文本：")


渲染后的训练文本：


In [66]:
text

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Retrieve the rate of biodegradation for a specific substance\ntype get_biodegradation_rate = (_: {\n// The substance for which you want to retrieve the biodegradation rate (e.g., plastic, oil, pesticide)\nsubstance: string\n}) => any;\n\n// Calculate the CO2 emissions produced by a specific process\ntype calculate_co2_emissions = (_: {\n// T

接下来，我们抽取200条数据进行批量转化与快速微调效果验证：

In [56]:
from datasets import load_dataset, Dataset, Features, Value
import random, json, os

In [25]:
# 随机采样 200 条（可换成你想要的索引集合）
seed = 42
random.seed(seed)
idxs = random.sample(range(len(ds_full)), 200)
ds_sample = ds_full.select(idxs)

In [26]:
print(ds_sample)  # sanity check

Dataset({
    features: ['conversations', 'tools', 'task', 'category', 'source', 'scenario_category'],
    num_rows: 200
})


In [109]:
ds_sample[0]

{'conversations': [{'from': 'system',
   'value': 'You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering. You should enclose your thoughts and internal monologue inside <think> </think> tags, and then provide your solution or response to the problem.\n\nYou are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. If available tools are not relevant in assisting with user query, just respond in natural conversational language. Don\'t make assumptions about what values to plug into functions. After calling & executing the functions, you will be provided with function results within <tool_response> </tool_response> XML tags. Here are the available tools:\n<tools>\n[\n{"name": "is_hotel_available", "descripti

In [93]:
# 注意：不要返回列名 'tools'，用新名字避免与原始列冲突
def map_convert_to_strings(ex):
    out = hermes_to_gptoss_example(ex)  # -> {"messages": [...], "tools": [...]}
    return {
        "messages_json": json.dumps(out["messages"], ensure_ascii=False),
        "tools_json": json.dumps(out["tools"], ensure_ascii=False),
    }

# 显式声明 features 为 string，杜绝 Arrow 猜类型
features = Features({
    "messages_json": Value("string"),
    "tools_json": Value("string"),
})

# 移除原始所有列，只保留我们需要的两列，避免同名冲突
ds_conv = ds_sample.map(
    map_convert_to_strings,
    remove_columns=ds_sample.column_names,
    features=features,
)
print(ds_conv[0])

Map: 100%|██████████| 200/200 [00:00<00:00, 2510.78 examples/s]

{'messages_json': '[{"role": "developer", "content": "You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering."}, {"role": "user", "content": "I want to book a hotel in Berlin from 3/15/2022 to 3/20/2022. Is the Queens Hotel available?"}, {"role": "assistant", "tool_calls": [{"function": {"name": "is_hotel_available", "arguments": {"hotel": "Queens Hotel", "city": "Berlin, Germany", "checkin": "2022-03-15", "checkout": "2022-03-20"}}}], "thinking": "Okay, the user wants to check the availability of the Queens Hotel in Berlin from March 15th to March 20th, 2022. Let me start by recalling the available tools. The only tool provided here is the hotel availability checker, which is called \\"is_hotel_available\\". \\n\\nSo, I need to construct the proper function call for that tool. The parameters required are hotel, city, che

In [94]:
ds_conv[1]

{'messages_json': '[{"role": "developer", "content": "You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering."}, {"role": "user", "content": "I am in New York City and I am looking for a restaurant within 500 meters. Can you help me?"}, {"role": "assistant", "tool_calls": [{"function": {"name": "find_nearby_restaurants", "arguments": {"location": "New York City", "distance": 500}}}], "thinking": "Okay, the user is asking for nearby restaurants in New York City within 500 meters. Let me think about how to approach this. The available tools include find_nearby_restaurants. So the function find_nearby_restaurants takes a location and a distance as parameters. Since the user is in NYC, I need to specify the location as \\"New York City\\" and set the distance to 500 meters.\\n\\nWait, the function parameters require the locat

In [95]:
save_path = "hermes_200_gptoss.jsonl"
with open(save_path, "w", encoding="utf-8") as f:
    for ex in ds_conv:
        f.write(json.dumps({
            "messages_json": ex["messages_json"],
            "tools_json": ex["tools_json"],
        }, ensure_ascii=False) + "\n")
print("Saved to:", os.path.abspath(save_path))

Saved to: /root/autodl-tmp/hermes_200_gptoss.jsonl


In [96]:
tokenizer

PreTrainedTokenizerFast(name_or_path='./gpt-oss-20b-bf16', vocab_size=199998, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|return|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	199998: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	199999: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200000: AddedToken("<|reserved_200000|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200001: AddedToken("<|reserved_200001|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200002: AddedToken("<|return|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200003: AddedToken("<|constrain|>", rstrip=False, 

In [23]:
def _fix_tools_for_template(tools):
    out = []
    for t in tools or []:
        if not isinstance(t, dict): 
            continue
        fn = t.get("function") if (t.get("type") == "function" and isinstance(t.get("function"), dict)) else t
        fn.setdefault("description", "")
        params = fn.get("parameters") or {}
        if isinstance(params, dict):
            if params.get("type") == "dict":
                params["type"] = "object"                     # ① dict -> object
            if params.get("type") == "object":
                props = params.setdefault("properties", {})
                if not isinstance(props, dict): 
                    params["properties"] = {}
                # 把外层 required 并进去
                if "required" not in params and isinstance(fn.get("required"), list):
                    params["required"] = fn["required"]       # ② 合并 required
                # 保证每个属性有 type/description
                for k, spec in list(props.items()):
                    if not isinstance(spec, dict): 
                        spec = {}
                    spec.setdefault("type", "string")
                    spec.setdefault("description", "")
                    props[k] = spec
        out.append({"type": "function", "function": fn})
    return out

def _fix_tool_messages(messages):
    for m in messages:
        if m.get("role") == "tool" and isinstance(m.get("content"), str):
            s = m["content"]
            try:
                m["content"] = json.loads(s)                  # ③ 字符串 -> 对象
            except Exception:
                try:
                    import ast
                    m["content"] = ast.literal_eval(s)
                except Exception:
                    pass
    return messages

def render_batch(batch):
    texts = []
    for m_str, t_str in zip(batch["messages_json"], batch["tools_json"]):
        messages = json.loads(m_str)
        tools = json.loads(t_str)

        messages = _fix_tool_messages(messages)
        tools = _fix_tools_for_template(tools)

        txt = tokenizer.apply_chat_template(
            messages,
            tools=tools,
            add_generation_prompt=False,
            reasoning_effort="low",
            tokenize=False,
        )
        texts.append(txt)
    return {"text": texts}


In [30]:
ds_text = ds_conv.map(render_batch, batched=True)

Map: 100%|██████████| 200/200 [00:00<00:00, 1927.02 examples/s]


In [31]:
ds_text

Dataset({
    features: ['messages_json', 'tools_json', 'text'],
    num_rows: 200
})

完整数据集地址：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250825200207894.png" alt="image-20250825200207894" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250825200241826.png" alt="image-20250825200241826" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/bff56b8959b8a17d9d0a6d1e72f5d9b8.png" alt="bff56b8959b8a17d9d0a6d1e72f5d9b8" style="zoom:50%;" />

我们抽取其中几条数据进行查看：

In [32]:
ds_text[0]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Checks the availability of a hotel for a given date range.\ntype is_hotel_available = (_: {\n// The name of the hotel.\nhotel: string,\n// The city where the hotel is located.\ncity: string,\n// The check-in date in the format "YYYY-MM-DD".\ncheckin: string,\n// The check-out date in the format "YYYY-MM-DD".\ncheckout: string\n}) => any;\n\n

In [115]:
ds_text[1]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Find nearby restaurants based on location\ntype find_nearby_restaurants = (_: {\n// The location to search for restaurants\nlocation: string,\n// The maximum distance in meters\ndistance: number\n}) => any;\n\n// Calculate the tip amount for a bill\ntype calculate_tip = (_: {\n// The amount of the bill\nbill_amount: number,\n// The percentag

In [116]:
ds_text[2]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Search for a given keyword in the English translation of the Quran and return the verses containing the keyword and total matches in JSON format.\ntype search_for_a_word_in_quran = (_: {\n// The keyword or phrase to search for in the Quran.\nsearchterm: string\n}) => any;\n\n// Calculates the CO2 emissions in kilograms from the use of public

In [117]:
ds_text[3]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Fetches and returns a list of all available stock symbols from the real-time-quotes API.\ntype stock_symbols = (_: {\n// If True, includes stock names in the returned data. Defaults to None.\nincludenames?: any, // default: "false"\n}) => any;\n\n// Fetch the Total Value Locked (TVL) for a specified DeFi protocol from the RapidAPI Toolbench.

In [119]:
ds_text[5]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Fetch a list of hosts that ran an EcoIndex analysis.\n\n    Args:\n        version (str): Engine version used for analysis (v0 or v1).\n        q (str, optional): Filter for partial host name.\n        date_from (str, optional): Start date for filtering (YYYY-MM-DD).\n        size (int, optional): Number of elements per page. Defaults to 50.

In [120]:
ds_text[6]['text']

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2025-08-25\n\nReasoning: low\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.\nCalls to these tools must go to the commentary channel: \'functions\'.<|end|><|start|>developer<|message|># Instructions\n\nYou are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.\n\n# Tools\n\n## functions\n\nnamespace functions {\n\n// Fetches the follower list for a specified user from the TikTok API.\ntype get_user_follower_list = (_: {\n// The ID of the user whose followers are to be fetched.\nuser_id: string,\n// The number of followers to retrieve, with a maximum of 200. Defaults to \'50\'.\ncount: string,\n// A timestamp indicating the starting point for the follower

## 【微调实战二】微调GPT-OSS模型Agent性能微调

&emsp;&emsp;当我们准备好了微调数据集之后，实际的微调过程并不会太复杂。

- **Step 1.读取数据集**

In [21]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="hermes_200_gptoss.jsonl", split="train")
print(dataset[0])   # 查看第一条

{'messages_json': '[{"role": "developer", "content": "You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering."}, {"role": "user", "content": "I want to book a hotel in Berlin from 3/15/2022 to 3/20/2022. Is the Queens Hotel available?"}, {"role": "assistant", "tool_calls": [{"function": {"name": "is_hotel_available", "arguments": {"hotel": "Queens Hotel", "city": "Berlin, Germany", "checkin": "2022-03-15", "checkout": "2022-03-20"}}}], "thinking": "Okay, the user wants to check the availability of the Queens Hotel in Berlin from March 15th to March 20th, 2022. Let me start by recalling the available tools. The only tool provided here is the hotel availability checker, which is called \\"is_hotel_available\\". \\n\\nSo, I need to construct the proper function call for that tool. The parameters required are hotel, city, che

- **Step 2.数据格式转化**

&emsp;&emsp;有了数据集之后，我们还需要将数据集转化为gpt-oss能够识别的数据格式，首先第一步是需要将其转化为gpt-oss的基本消息格式：

In [24]:
ds_text = dataset.map(render_batch, batched=True)

In [25]:
ds_text

Dataset({
    features: ['messages_json', 'tools_json', 'text'],
    num_rows: 200
})

其中'text'字段中就保存着可以用于微调的完整文本结构，其中assistant之前是输入，之后是输出：

In [26]:
print(ds_text[0]['text'])

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-25

Reasoning: low

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.<|end|><|start|>developer<|message|># Instructions

You are a deep thinking AI, you may use extremely long chains of thought to deeply consider the problem and deliberate with yourself via systematic reasoning processes to help come to a correct solution prior to answering.

# Tools

## functions

namespace functions {

// Checks the availability of a hotel for a given date range.
type is_hotel_available = (_: {
// The name of the hotel.
hotel: string,
// The city where the hotel is located.
city: string,
// The check-in date in the format "YYYY-MM-DD".
checkin: string,
// The check-out date in the format "YYYY-MM-DD".
checkout: string
}) => any;

} // namespace functions<|end|><

- **Step 3.LoRA参数灌注**

In [27]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.model` require gradients


基本参数说明如下：

| 参数                               | 含义                   | 推荐值/当前设置                        | 作用与说明                                                     |
| -------------------------------- | -------------------- | ------------------------------- | --------------------------------------------------------- |
| **r**                            | LoRA 秩（rank）         | 8（可选 8/16/32/64/128）            | 控制新增参数规模；值越大表达能力越强，但显存占用也随之上升。中文推理链推荐从 8/16 起步。           |
| **target\_modules**              | 注入的线性层               | q/k/v/o 投影 + MLP (gate/up/down) | 指定在哪些层挂 LoRA。覆盖注意力 + MLP → 更好适配中文句法与逻辑；若显存不足可只保留 q/k/v/o。 |
| **lora\_alpha**                  | LoRA 缩放系数            | 16                              | 放大 LoRA 分支输出，等效调整学习率。常用范围 16–32，r 小时可适当调高。                |
| **lora\_dropout**                | LoRA dropout 概率      | 0                               | 控制 LoRA 分支的随机丢弃率。0 在 Unsloth 中优化过，更快更省；若过拟合可尝试 0.05–0.1。  |
| **bias**                         | Bias 参数训练策略          | "none"                          | 是否对注入层 bias 可训练。"none" 开销最小且优化最好。                         |
| **use\_gradient\_checkpointing** | 梯度检查点                | "unsloth"                       | 节省 \~30% 显存，支持更长上下文和更大 batch；推荐开启。                        |
| **random\_state**                | 随机种子                 | 3407                            | 保证实验可复现；任意固定值均可。                                          |
| **use\_rslora**                  | Rank-Stabilized LoRA | False                           | 在大 r（≥64）时可提高稳定性；一般场景不必启用。                                |
| **loftq\_config**                | LoftQ 量化配置           | None                            | 结合量化训练 LoRA，在 4/8bit 推理时更稳。默认关闭；若目标是低比特部署，可配置启用。          |


- **Step 4.设置微调参数**

In [28]:
ds_text

Dataset({
    features: ['messages_json', 'tools_json', 'text'],
    num_rows: 200
})

In [29]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = ds_text,
    args = SFTConfig(
        max_seq_length = 4096,
        dataset_text_field = "text",   # 你上一步 map 生成了 text 列
        packing = False,               # 一般先关掉 packing，避免拼接样本
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="tensorboard",     # 📌 启用 TensorBoard
        logging_dir="./logs",        # 日志目录
        logging_strategy="steps",
        logging_steps=1,            # 记录频率
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 200/200 [00:01<00:00, 128.94 examples/s]


- **Step 5.开始微调**

In [30]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)
/root/miniconda3/envs/unsloth/lib/python3.12/site-packages/torch/_inductor/runtime/triton_helpers.py:150: UserWarning: Logical operators 'and' and 'or' are deprecated for non-scalar tensors; please use '&' or '|' instead
  mask |= a_isnan and not b_isnan
/root/miniconda3/envs/unsloth/lib/python3.12/site-packages/torch/_inductor/runtime/triton_helpers.py:150: UserWarning: Logical oper

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.278600
2,3.128200
3,3.592000
4,3.236400
5,3.003900
6,2.331300
7,2.417000
8,2.222700
9,1.841600
10,1.481800


- **Step 7.微调后模型导出**

最后，我们还可以将模型合并导出，以便下次使用：

In [31]:
model

PeftModel(
  (base_model): LoraModel(
    (model): GptOssForCausalLM(
      (model): GptOssModel(
        (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
        (layers): ModuleList(
          (0-23): 24 x GptOssDecoderLayer(
            (self_attn): GptOssAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2880, out_features=4096, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2880, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
        

In [32]:
model.save_pretrained_merged(save_directory = "GPT-OSS-finetuned", 
                             tokenizer = tokenizer, 
                             save_method = "merged_16bit")

Detected local model directory: ./gpt-oss-20b-bf16
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Copying safetensors from local directory: ./gpt-oss-20b-bf16
Copied safetensors index file from local model


Unsloth: Merging weights into 16bit:   0%|          | 0/9 [00:00<?, ?it/s]

Copied model-00008-of-00009.safetensors from local model directory


Unsloth: Merging weights into 16bit:  11%|█         | 1/9 [00:10<01:20, 10.00s/it]

Copied model-00004-of-00009.safetensors from local model directory


...

Unsloth: Merging weights into 16bit:  78%|███████▊  | 7/9 [01:06<00:18,  9.46s/it]

Copied model-00006-of-00009.safetensors from local model directory


Unsloth: Merging weights into 16bit:  89%|████████▉ | 8/9 [01:15<00:09,  9.41s/it]

Copied model-00009-of-00009.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 9/9 [01:19<00:00,  8.88s/it]


<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250825204654903.png" alt="image-20250825204654903" style="zoom:50%;" />

- **Step 8.微调效果对比**

接下来即可使用vllm对其进行调用，并借助EvalScope进行测试。需要借助vLLM调用导出后的模型，然后借助evalscope进行测试。

```python
from evalscope import TaskConfig, run_task
task_cfg = TaskConfig(
    model='./GPT-OSS-finetuned',
    api_url='http://127.0.0.1:8000/v1/chat/completions',
    eval_type='service',
    datasets=[
        'data_collection',
    ],
    dataset_args={
        'data_collection': {
            'dataset_id': 'modelscope/EvalScope-Qwen3-Test',
            'filters': {'remove_until': '</think>'}  # 过滤掉思考的内容
        }
    },
    eval_batch_size=128,
    generation_config={
        'max_tokens': 30000,  # 最大生成token数，建议设置为较大值避免输出截断
        'temperature': 0.6,  # 采样温度 (qwen 报告推荐值)
        'top_p': 0.95,  # top-p采样 (qwen 报告推荐值)
        'top_k': 20,  # top-k采样 (qwen 报告推荐值)
        'n': 1,  # 每个请求产生的回复数量
    },
    timeout=60000,  # 超时时间
    stream=True,  # 是否使用流式输出
    limit=2000,  # 设置为2000条数据进行测试
)

run_task(task_cfg=task_cfg)
```

测试后即可在前端中观察测评报告：
```bash
evalscope app
```

原始模型评分：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/dc35abae0cad62215355b8640f2681e0.png" alt="dc35abae0cad62215355b8640f2681e0" style="zoom:50%;" />

微调后模型评分：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/403e787b08e9472d59915854d436f480.png" alt="403e787b08e9472d59915854d436f480" style="zoom:50%;" />

两个模型对比：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/acfc8a3144f201ab63cad576a73ec7d0.png" alt="acfc8a3144f201ab63cad576a73ec7d0" style="zoom:50%;" />